In [2]:
import sys
sys.path.append('../src/')

from benchmark.tools import *
from benchmark.models import Detector
from benchmark.utils import TsvHandle, run_detection

import wider_dataset
# https://github.com/borhanMorphy/fastface
import fastface as ff

ModuleNotFoundError: No module named 'fastface'

In [ ]:
train, test, validation = tfds.load('wider_dataset', split=['train', 'test', 'validation'])
data = validation

In [ ]:
class FastFaceDetector(Detector):
    def __init__(self):
        model_weights = "./lffd_original.ckpt"
        self.detector = ff.FaceDetector.from_pretrained("./lffd_original.ckpt")
        self.detector.summarize()
    
    def detect(self, image):
        try:
            predicted, = self.detector.predict(image, det_threshold=.8, iou_threshold=.4)
            return predicted['boxes'], predicted['scores']
        except Exception:
            return [], []
    
fast_face = FastFaceDetector()

In [ ]:
with TsvHandle('fastface_filtered') as tsv:
    run_detection(tsv, data, fast_face, None, False, False, None)